<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing and Evaluating a LlamaIndex OpenAI Agent Application</h1>

With the new OpenAI API that supports function calling, it’s never been easier to build your own agent.

In this notebook tutorial, we showcase how to write your own OpenAI agent in under 50 lines of code and use Phoenix to inspect the internals of the Agent. It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

Install LlamaIndex and other dependencies.

In [1]:
!pip install "openai>=1" "arize-phoenix[llama-index]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 23.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.3/929.3 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 32.8 MB/s eta 0:00:00
  Using cache

Import libraries.

In [2]:
import os
from getpass import getpass

import openai
import pandas as pd
import phoenix as px
from llama_index.agent import OpenAIAgent
from llama_index.callbacks import CallbackManager
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT
from llama_index.tools import FunctionTool
from phoenix.trace.llama_index import (
    OpenInferenceTraceCallbackHandler,
)

pd.set_option("display.max_colwidth", 1000)

You can run Phoenix in the background to collect trace data emitted by any LlamaIndex application that has been instrumented with the `OpenInferenceTraceCallbackHandler`.

Launch Phoenix and follow the instructions in the cell output to open the Phoenix UI (the UI should be empty because we have yet to run a LlamaIndex application).

In [3]:
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://ndmcdelk8881-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Let’s start by importing some building blocks and defining tools that our agent can use

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

Provide your API keys to access Open AI

In [5]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


Now, we define our agent that’s capable of holding a conversation and calling tools.

The meat of the agent logic is in the chat method. At a high-level, there are 3 steps:

- Call OpenAI to decide which tool (if any) to call and with what arguments.

- Call the tool with the arguments to obtain an output

- Call OpenAI to synthesize a response from the conversation context and the tool output.

The reset method resets the conversation context, so we can start another conversation.

For fun, let's make the agent chat in the style of Shakespeare.

In [6]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
callback_handler = OpenInferenceTraceCallbackHandler()
callback_manager = CallbackManager(handlers=[callback_handler])
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    callback_manager=callback_manager,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

Let's now chat with our agent!

In [7]:
response = agent.query("What is (121 * 3) + 42?")
print(response)

The result of (121 * 3) + 42 is 405.


Let's chat with our agent a few more times. This time with some follow-up questions.

In [8]:
queries = [
    "What is (121 * 3) + 42?",
    "what is 3 * 3?",
    "what is 4 * 4?",
    "what is 75 * (3 + 4)?",
    "what is 23 times 87",
]

for query in queries:
    print(f"> {query}")
    response = agent.query(query)
    print(response)
    agent.reset()
    print("---")

> What is (121 * 3) + 42?
The result of (121 * 3) + 42 is 405.
---
> what is 3 * 3?
The product of 3 multiplied by 3 is 9.
---
> what is 4 * 4?
Four times four equals sixteen.
---
> what is 75 * (3 + 4)?
The result of 75 multiplied by the sum of 3 and 4 is 525.
---
> what is 23 times 87
The product of 23 times 87 is 2001.
---


Open the `session.url` in your browser to take a look at the traces in Phoenix. Note that LLM spans contain the OpenAI function calls, and that we can inspect what tool the LLM picked based on the queries.

To learn more about function calling, check out the [OpenAI API docs](https://openai.com/blog/function-calling-and-other-api-updates).


In [9]:
print(f"Open the Phoenix UI if you haven't already: {session.url}")

Open the Phoenix UI if you haven't already: https://ndmcdelk8882-496ff2e9c6d22116-6006-colab.googleusercontent.com/


We can also inspect the agent's chat history as a dataframe.

In [10]:
ds = px.TraceDataset.from_spans(list(callback_handler.get_spans()))
ds.dataframe.head()

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,conversation,context.trace_id,...,attributes.llm.input_messages,attributes.llm.model_name,attributes.llm.invocation_parameters,attributes.llm.output_messages,attributes.llm.token_count.prompt,attributes.llm.token_count.completion,attributes.llm.token_count.total,attributes.tool.name,attributes.tool.description,attributes.tool.parameters
0,agent_step,AGENT,None,2023-12-20 00:52:13.536324+00:00,2023-12-20 00:52:16.393546+00:00,OK,,[],None,440273de-1e26-495a-9b09-36b9c1fadd39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,llm,LLM,0342a3bf-09fe-4bf7-833c-59781b2af1eb,2023-12-20 00:52:15.443956+00:00,2023-12-20 00:52:16.393426+00:00,OK,,[],None,440273de-1e26-495a-9b09-36b9c1fadd39,...,"[{'message.role': 'system', 'message.content': 'You are a Shakespearean writing assistant who speaks in a Shakespearean style. You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like ""thou"" and ""hath”. Here are some example of Shakespeare's style: - Romeo, Romeo! Wherefore art thou Romeo? - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind. - Shall I compare thee to a summer's day? Thou art more lovely and more temperate. '}, {'message.role': 'user', 'message.content': 'What is (121 * 3) + 42?'}, {'message.role': 'assistant', 'message.content': None, 'message.tool_calls': [{'tool_call.function.name': 'multiply', 'tool_call.function.arguments': '{\n ""a"": 121,\n ""b"": 3\n}'}]}, {'message.role': 'tool', 'message.content': '363', 'message.name': 'multiply'}, {'message.role': 'assistant', 'message.content': None, 'message.tool_calls': [{'tool_...",gpt-3.5-turbo-0613,"{""model"": ""gpt-3.5-turbo-0613"", ""temperature"": 0.1, ""max_tokens"": null}","[{'message.role': 'assistant', 'message.content': 'The result of (121 * 3) + 42 is 405.'}]",292.0,17.0,309.0,NaN,NaN,NaN
2,function_call,TOOL,0342a3bf-09fe-4bf7-833c-59781b2af1eb,2023-12-20 00:52:15.443622+00:00,2023-12-20 00:52:15.443725+00:00,OK,,[],None,440273de-1e26-495a-9b09-36b9c1fadd39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,add,"add(a: int, b: int) -> int\nAdd two integers and returns the result integer","{'title': 'add', 'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}"
3,llm,LLM,0342a3bf-09fe-4bf7-833c-59781b2af1eb,2023-12-20 00:52:14.560922+00:00,2023-12-20 00:52:15.443354+00:00,OK,,[],None,440273de-1e26-495a-9b09-36b9c1fadd39,...,"[{'message.role': 'system', 'message.content': 'You are a Shakespearean writing assistant who speaks in a Shakespearean style. You help people come up with creative ideas and content like stories, poems, and songs that use Shakespearean style of writing style, including words like ""thou"" and ""hath”. Here are some example of Shakespeare's style: - Romeo, Romeo! Wherefore art thou Romeo? - Love looks not with the eyes, but with the mind; and therefore is winged Cupid painted blind. - Shall I compare thee to a summer's day? Thou art more lovely and more temperate. '}, {'message.role': 'user', 'message.content': 'What is (121 * 3) + 42?'}, {'message.role': 'assistant', 'message.content': None, 'message.tool_calls': [{'tool_call.function.name': 'multiply', 'tool_call.function.arguments': '{\n ""a"": 121,\n ""b"": 3\n}'}]}, {'message.role': 'tool', 'message.content': '363', 'message.name': 'multiply'}]",gpt-3.5-turbo-0613,"{""model"": ""gpt-3.5-turbo-0613"", ""temperature"": 0.1, ""max_tokens"": null}","[{'message.role': 'assistant', 'message.tool_calls': [{'tool_call.function.name': 'add', 'tool_call.function.arguments': '{\n ""a"": 363,\n ""b"": 42\n}'}]}]",263.0,21.0,284.0,NaN,NaN,NaN
4,function_call,TOOL,0342a3bf-09fe-4bf7-833c-59781b2af1eb,2023-12-20 00:52:14.560458+00:00,2023-12-20 00:52:14.560676+00:00,OK,,[],None,440273de-1e26-495a-9b09-36b9c1fadd39,...,NaN,NaN,NaN,N